In [1]:
import os

In [2]:
%pwd

'd:\\Aiprojects\\Textsummarization\\text-summarization\\Notebooks'

In [3]:
os.chdir("../")
%pwd

'd:\\Aiprojects\\Textsummarization\\text-summarization'

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ModelEvaluationConfig:

    model_path: Path
    data_path: Path
    root_dir: Path
    tokenizer_path: Path
    metric_filename: Path


In [5]:
from src.constants import * 
from src.utils import read_yaml , create_directory

In [6]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH , params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])
    def get_evaluation_config(self) ->ModelEvaluationConfig:
        config = self.config.model_evaluation
        create_directory([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            model_path=config.model_path,
            data_path=config.data_path,
            root_dir=config.root_dir,
            tokenizer_path=config.tokenizer_path,
            metric_filename=config.metric_filename
        )
        return model_evaluation_config


In [7]:
!pip install evaluate

In [8]:
import torch
import sys 
from datasets import load_from_disk,load_dataset
import evaluate
from transformers import AutoTokenizer , AutoModelForSeq2SeqLM
import pandas as pd
from tqdm import tqdm


d:\Aiprojects\Textsummarization\text-summarization\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class ModelEvaluation:
    
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config


    def generate_batch_chuncks(self,full_data,batch_size):

        for i in range(0,len(full_data),batch_size):
            yield full_data[i:i+batch_size]


    def calculate_metrics_test(self,dataset,metric,tokenizer,model, input_name, target_name,device ,batch_size = 16 ):

        
        input_batches = list(self.generate_batch_chuncks(dataset[input_name],batch_size))
        target_batches = list(self.generate_batch_chuncks(dataclass[target_name],batch_size))

        for input_batch , target_batch in tqdm(zip(input_batches,target_batches),total=len(input_batches)):
            inputs = tokenizer(input_batch, max_length=1024, truncation=True, 
                   padding="max_length", return_tensors="pt")
            
            summary = model.generate(input_ids=inputs["input_ids"].to(device),
                           attention_mask=inputs["attention_mask"].to(device), 
                           length_penalty=0.8, num_beams=8, max_length=128)
            
            decoded_summary = [tokenizer.decode(s, skip_special_tokens=True, 
                                      clean_up_tokenization_spaces=True) 
                     for s in summary]      
            decoded_summary = [d.replace("", " ") for d in decoded_summary]

            metric.add_batch(predictions=decoded_summary, references=target_batch)
        
        final_score = metric.compute()
        return final_score
    
    def evaluate(self):

        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        rouge = evaluate.load("rouge")
        
        dataset_pt = load_from_disk(self.config.data_path)
        input_name = 'dialogue'
        target_name = 'summary'

        scores = self.calculate_metrics_test(
            dataset=dataset_pt['test'][0:300] , metric = rouge , tokenizer=tokenizer , 
            model=model,input_name = input_name,target_name = target_name ,device  = device 
        )

        rouge_metrics = ['rouge1','rouge2','rougeL','rougeLsum']

        rouge_dict = dict((metric, scores[metric].mid.fmeasure ) for metric in rouge_metrics )

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_filename, index=False)

        
        

In [10]:

    config = ConfigurationManager()
    model_evaluation_config = config.get_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()



AttributeError: BartTokenizerFast has no attribute to